# 👋 Got GenUI? If not, welcome to a quick tutorial!

### Veronica Ward and John Maeda walk you through a quick tour of how easy it is to "print code" and generate meaningful app experiences in September 2024 with just gpt-4o out of the box. 

### 🌱 Prerequisites

You'll need to have:

1. An Azure AI account
2. An Azure OpenAI model deployed (preferably gpt-4o)

### 📦 Get the necessary pkgs and base code

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.4"
#r "nuget: Azure.Core, 1.42.0"
#r "nuget: Azure.Identity, 1.13.0-beta.1"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.18.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Core, 1.18.1-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.1-alpha"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

### 🔑 And all your keys together

My config.json looks like this:
```
{
  "type": "azure",
  "model": "gpt-4o",
  "endpoint": "https://ai-johnmaeda.../",
  "apikey": "...",
  "org": ""
}
```

In [ ]:
#!import ../config/MaedaSettings.cs
#!import ../config/Utils.cs

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using System;
using System.Text.Json;
using System.Collections.Generic;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Net.Http;
using System.Net.Http.Headers;
using Microsoft.Extensions.DependencyInjection;
using System.Threading;
using OpenAI.Files;
using OpenAI.VectorStores;

var settingsPath = Path.GetFullPath("../config/settings.json");
Console.WriteLine($"Settings Path: {settingsPath}");

var maedaSettings = new MaedaSettings(settingsPath, true);

// Load all settings into a tuple
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = maedaSettings.LoadSettings();

### 🔥 Semantic Kernel always helps :-)

In [3]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

IKernelBuilder builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
                deploymentName: model,
                endpoint: azureEndpoint,
                apiKey: apiKey);

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

OpenAIClientProvider provider = OpenAIClientProvider.ForAzureOpenAI(apiKey, new Uri(azureEndpoint));

### 🧑‍🍳 Use my little agent wrapper

In [4]:
#!import ../config/Utils.cs
#!import ../config/MaedaAgentManager.cs

#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

MaedaAgentManager agentManager = new MaedaAgentManager(kernel, provider);


# 🖨️ Let's print a P5 JS app!

Here we're giving the agent a backstory and instructions.

In [43]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

string agentName = "genuip5js";
string description = """
  Processing p5js programmer that takes ideas and writes 
  complete code for them.
""";
string instructions = """
  Write a program in p5js in response to the user and only 
  respond with a complete HTML file with JS for p5js included.
""";

string genuip5jsId = await agentManager.CreatePlainAgent(agentName, model, description, instructions);

OpenAIAssistantAgent genuip5js = await OpenAIAssistantAgent.RetrieveAsync(kernel, provider, genuip5jsId);

Next we give it the direction for what kind of p5js app to make.

In [44]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

var question = """
Write a program that simulates snow falling from the sky.
The snowflakes should be white circles that fall from the top of the screen.
The snowflakes should avoid the position of the mouse with a wide avoidance radius of 150 pixels.
The snowflakes should swirl like a cyclone on screen.
""";

var threadId = await genuip5js.CreateThreadAsync();
await genuip5js.AddChatMessageAsync(threadId, new ChatMessageContent(AuthorRole.User, question));

// Retrieve the response.
string responseText = "";
await foreach (ChatMessageContent content in genuip5js.InvokeAsync(threadId))
{
    if (content.Role != AuthorRole.Tool)
    {
        responseText += content.Content;
    }
}

We then store the output to a file.

In [ ]:
var app_p5jsHtml = Path.GetFullPath("app_p5js.html");
// remove the first and last lines of the response
responseText = responseText.Substring(responseText.IndexOf('\n') + 1);

File.WriteAllText(app_p5jsHtml, responseText);
Console.WriteLine($"App HTML file saved to: {app_p5jsHtml}");

And open it up in our browser.

In [46]:
// execute a system command to open the html File that doesn't have permission problems on a macos
var process = System.Diagnostics.Process.Start("open", app_p5jsHtml);

### 🧽 Let's do some cleanup

Delete the thread we made.

In [47]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001
bool result = await genuip5js.DeleteThreadAsync(threadId);

Get rid of the agent we created.

In [ ]:
await agentManager.DeleteAllAgentsAsync();

# 🖨️ Print a Bootstrap React app

We can now pursue writing React code. Ready? Set! Go!

The model I'm using seems to like Bootstrap React so let's go with that.

In [37]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

string reactKit = "Bootstrap React";

string genuireactjsId = await agentManager.CreatePlainAgent(
    // name
    "genuireactjs", 
    model, 
    // description
    """
    Skilled React frontend dev that takes UI ideas and writes 
    complete code for them using 
    """ + reactKit + " components.",
    // instructions
    """
    Writes working code in response to the user and only 
    responds with a complete HTML file with JS for
    """ + reactKit + " included."
    );

OpenAIAssistantAgent genuireactjs = await OpenAIAssistantAgent.RetrieveAsync(kernel, provider, genuireactjsId);

Define the goal of the app screens you wish to create.

In [38]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001

var threadId = await genuireactjs.CreateThreadAsync();
await genuireactjs.AddChatMessageAsync(threadId, 
    new ChatMessageContent(AuthorRole.User, 
    """
    Write a program with a login screen that transitions
    to a landing page welcoming the user to a new kind of
    AI-based chat system. The landing page should let the
    user input a chat and then have a default response back
    of 'Hello, how can I help you today?' and to display
    what the user has entered.
    """
));

// Retrieve the response.
string responseText = "";
await foreach (ChatMessageContent content in genuireactjs.InvokeAsync(threadId))
{
    if (content.Role != AuthorRole.Tool)
    {
        responseText += content.Content;
    }
}

Store the file to view in your browser.

In [ ]:
var app_reactjsHtml = Path.GetFullPath("app_bootstrap_reactjs.html");
// remove the first and last lines of the response
responseText = responseText.Substring(responseText.IndexOf('\n') + 1);

File.WriteAllText(app_reactjsHtml, responseText);
Console.WriteLine($"App HTML file saved to: {app_reactjsHtml}");

Open the file in your browser.

In [40]:
// execute a system command to open the html File that doesn't have permission problems on a macos
var process = System.Diagnostics.Process.Start("open", app_reactjsHtml);

### 🧽 Cleanup time

Delete the thread you created.

In [41]:
#pragma warning disable SKEXP0110, SKEXP0001, SKEXP0050, CS8600, CS8604, OPENAI001
bool result = await genuireactjs.DeleteThreadAsync(threadId);

And delete the assistant set up in the cloud.

In [ ]:
await agentManager.DeleteAllAgentsAsync();